In [1]:
import numpy as np
import pandas as pd
from pytorch_forecasting import TimeSeriesDataSet

/home/volody/code/study-py/ts-pytorch/.venv/lib/python3.11/site-packages/lightning_utilities/core/imports.py:14: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources


In [2]:
mvtseries = pd.read_csv(
    "../assets/datasets/time_series_smf1.csv",
    parse_dates=["datetime"],
    index_col="datetime",
)

stat_by_variable = {
    "Incoming Solar": "sum",
    "Wind Dir": "mean",
    "Snow Depth": "sum",
    "Wind Speed": "mean",
    "Dewpoint": "mean",
    "Precipitation": "sum",
    "Vapor Pressure": "mean",
    "Relative Humidity": "mean",
    "Air Temp": "max",
}

mvtseries = mvtseries.resample("D").agg(stat_by_variable)
mvtseries = mvtseries.ffill()

mvtseries.to_csv("../assets/daily_multivariate_timeseries.csv")

In [3]:
mvtseries.tail()

,Incoming Solar,Wind Dir,Snow Depth,Wind Speed,Dewpoint,Precipitation,Vapor Pressure,Relative Humidity,Air Temp
datetime,,,,,,,,,
2013-09-26,1034.2,137.083333,0.0,0.550000,1.645833,12.3,689.458333,0.970000,3.9
2013-09-27,2182.3,108.958333,0.0,0.608333,0.529167,0.0,634.625000,0.850000,8.3
2013-09-28,3384.5,97.250000,0.0,0.708333,0.670833,0.0,642.125000,0.606250,16.5
2013-09-29,478.2,108.666667,0.0,1.037500,2.129167,0.6,719.083333,0.486250,13.9
2013-09-30,2554.8,115.250000,0.0,0.962500,4.787500,2.0,863.416667,0.747083,12.2


In [16]:
TARGET = "Incoming Solar"
N_LAGS = 3
HORIZON = 1

input_data = []
output_data = []
for i in range(N_LAGS, mvtseries.shape[0] - HORIZON + 1):
    input_data.append(mvtseries.iloc[i - N_LAGS : i].values)
    output_data.append(mvtseries.iloc[i : (i + HORIZON)][TARGET])

input_data, output_data = np.array(input_data), np.array(output_data)
input_data.shape, output_data.shape

((2189, 3, 11), (2189, 1))

In [17]:
mvtseries.T.head(5)

datetime,2007-10-01,2007-10-02,2007-10-03,2007-10-04,2007-10-05,2007-10-06,2007-10-07,2007-10-08,2007-10-09,2007-10-10,...,2013-09-21,2013-09-22,2013-09-23,2013-09-24,2013-09-25,2013-09-26,2013-09-27,2013-09-28,2013-09-29,2013-09-30
Incoming Solar,1381.500000,3953.200000,3098.100000,2213.900000,1338.800000,3671.500,4193.700000,4213.800000,4195.500000,1340.200000,...,2581.000000,1592.000000,4113.600000,2134.200000,1250.200000,1034.200000,2182.300000,3384.500000,478.200000,2554.8000
Wind Dir,281.583333,173.416667,263.875000,209.227273,234.409091,248.375,238.130435,159.583333,128.833333,224.291667,...,104.875000,144.833333,51.888889,51.888889,157.095238,137.083333,108.958333,97.250000,108.666667,115.2500
Snow Depth,0.000000,0.000000,0.000000,0.000000,0.000000,0.000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0000
Wind Speed,0.604167,0.595833,0.758333,0.591667,0.795833,0.800,0.645833,0.629167,0.845833,0.670833,...,0.795833,0.662500,0.654167,0.641667,0.529167,0.550000,0.608333,0.708333,1.037500,0.9625
Dewpoint,1.145833,-0.704167,0.541667,-0.754167,-2.558333,-4.525,-0.404167,-2.583333,-5.325000,-2.583333,...,0.112500,3.554167,2.433333,2.529167,1.104167,1.645833,0.529167,0.670833,2.129167,4.7875


In [18]:
mvtseries["time_index"] = np.arange(mvtseries.shape[0])
mvtseries["group_id"] = 0
mvtseries.tail()

,Incoming Solar,Wind Dir,Snow Depth,Wind Speed,Dewpoint,Precipitation,Vapor Pressure,Relative Humidity,Air Temp,time_index,group_id
datetime,,,,,,,,,,,
2013-09-26,1034.2,137.083333,0.0,0.550000,1.645833,12.3,689.458333,0.970000,3.9,2187,0
2013-09-27,2182.3,108.958333,0.0,0.608333,0.529167,0.0,634.625000,0.850000,8.3,2188,0
2013-09-28,3384.5,97.250000,0.0,0.708333,0.670833,0.0,642.125000,0.606250,16.5,2189,0
2013-09-29,478.2,108.666667,0.0,1.037500,2.129167,0.6,719.083333,0.486250,13.9,2190,0
2013-09-30,2554.8,115.250000,0.0,0.962500,4.787500,2.0,863.416667,0.747083,12.2,2191,0


In [19]:
# create the dataset from the pandas dataframe
dataset = TimeSeriesDataSet(
    data=mvtseries,
    group_ids=["group_id"],
    target="Incoming Solar",
    time_idx="time_index",
    max_encoder_length=7,
    max_prediction_length=1,
    time_varying_unknown_reals=[
        "Incoming Solar",
        "Wind Dir",
        "Snow Depth",
        "Wind Speed",
        "Dewpoint",
        "Precipitation",
        "Vapor Pressure",
        "Relative Humidity",
        "Air Temp",
    ],
)

In [20]:
# convert the dataset to a dataloader
data_loader = dataset.to_dataloader(batch_size=1, shuffle=False)

x, y = next(iter(data_loader))

x["encoder_cont"]
y


(tensor([[4213.7998]]), None)